In [1]:
import os

# Python exceptions

Many modern langauges such as Python, Java, C# include an error handling mechanism called exceptions, and supporting syntax called the "try/catch" or "try/except" block.

"Exceptions" are named very appropriately, they indicate that something unexpected or _exceptional_ has taken place inside a function. In other words, something is wrong!

Take a look at this function:

```python
def read_file(filename):
    f = open(filename) <== What if the file doesn't exist?? Will the whole program die?
    return f.readlines()
```

In languages without exceptions, you might do defensive programming as such:
```python
def read_file(filename):
    
    if not os.path.isfile(filename):
        return -1 <== Negative code indicates that this function did not execute correctly
    
    f = open(filename)
    return f.readlines()
```

Exceptions allow us to manage errors in a safer and more rigorous manner:

In [2]:
def read_file(filename):
    try:
        f = open(filename)
        return f.readlines()
    except:
        print("Horrible error, the file wasn't found")
        return ""

In [3]:
read_file("nofile.txt")

Horrible error, the file wasn't found


''

Exceptions allow us to catch specific error scenarios:

In [9]:
def read_file2(filename):
    try:
        f = open(filename)
        return f.readlines()
    except FileNotFoundError:
        print("Horrible error, the file wasn't found")
        return ""
    except PermissionError:
        print("The file exists but you do not have access to access it")
        return ""

In [10]:
read_file2('nofile.txt')

Horrible error, the file wasn't found


''

**Note** Here is an unexpected pro-tip, you generally should not "catch" exceptions. For example, if the file exist, you can't do anything about it anyway, just let the program crash!

In [4]:
def read_file3(filename):
    f = open(filename)
    return f.readlines()

In [6]:
read_file3('nofile.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'nofile.txt'

### try...except...else
Python provides an `else` block, which executes if NO exception was raised

In [28]:
def read_file4(filename):
    try:
        f = open(filename)
        rslt= f.readlines()
    except FileNotFoundError:
        print("Horrible error, the file wasn't found")
        return ""
    except PermissionError:
        print("The file exists but you do not have access to access it")
        return ""
    else:
        print("File read correctly")
    return rslt

In [29]:
read_file4('../../postcell.conf')

File read correctly


['{\n',
 '    "class_id": "2024_quarter1_saturday_advancedpythonfords",\n',
 '    "instructor_id": "doJIH2jibYWOOJyGm6zsDNe93722",\n',
 '    "should_send_to_server": "true",\n',
 '    "student_id": "Shahbaz_Chaudhary",\n',
 '    "url": "https://postcell.io/post_cell"\n',
 '}']

Realistically there aren't very many useases for the `try ... else` clause. Most languages which provide exception handling don't provide a `try/else` clause. 

### try...except...finally
Unlike the `try/else` clause, the `try/finally` clause is extremely important. It is sometimes necessary to cleanup resource which were acquired during a try block.

In [36]:
def read_files5(file1, file2):
    try:
        f1, f2 = None, None
        f1 = open(file1)
        f2 = open(file2)
    except FileNotFoundError:
        print("Horrible error, the file wasn't found")
    finally:
        print("Closing open files")
        if f1: f1.close()
        if f2: f2.close()

Notice which part of the try/except/finally block gets called if an exception is raised or isn't raised

In [37]:
read_files5('nofile.txt', 'nofile.csv')

Horrible error, the file wasn't found
Closing open files


In [38]:
read_files5('../../postcell.conf', '../../postcell.conf')

Closing open files


### Raise exceptions

If you are writing a function or a full library, you may have to raise exceptions yourself. The syntax is very simple:

In [11]:
def get_lottery_number(password):
    LOTTERY_NUM = 42
    if password == 123:
        return LOTTERY_NUM
    else:
        raise PermissionError("Bad password :( ")

In [12]:
get_lottery_number(456)

PermissionError: Bad password :( 

### Custom exceptions
Creating custom exceptions is the domain of software engineers. As a data scientist, you should never have to create your own exceptions. If you even do end up writing a library, the syntax is as follows:

In [17]:
class ConstraintViolated(Exception):
    def __init__(self, message):
        super().__init__(message)

In [21]:
class Professor():
    def __init__(self, name, department, salary):
        self.name = name
        self.department = department
        self.salary = salary
        
        if salary == 0:
            raise ConstraintViolated("Professor salary must not be zero dollar")

In [22]:
Professor("Shahbaz", "PSD", 0)

ConstraintViolated: Professor salary must not be zero dollar

### But why do we need exceptions?

In the example, the class simply won't be created if an exception is thrown. A less obvious example is why you would raise an exception inside a function.

For example, why does this cause an exception:

In [24]:
open('nofile.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'nofile.txt'

Why not just return an empty string or None (as some functions do).
The reason has to do with how to _think_ about these functions. If the `open` function returned an empty string, does that mean the file was empty or that the file didn't exist?

If that function returned None, that can be an indicator that an error occured, but wouldn't we want to know more fine-grained detail about the cause of the error: the file doesn't exist or the user doesn't have access to it?

### Checked exceptions (which Python doesn't have)

Python is a dynamic language, which means it does not have strict type checking. You can simply assign a value to a variable `gpa = 3.5`, without needing to give an explicit type. Some languages not only requier types, `double gpa = 3.5`, they even have strict exception checking!

For example, if Python was like Java, the following code would not work:

```python
def count_lines(filename):
    file = open(filename)
    return len(file.readlines())
```

Java, at compile time, would recognize that the `open` command can throw a `FileNotFound` exception. It would force us to acknowledge or handle the exception. New programmers might do something like this:

```python
def count_lines(filename):
    try:
        file = open(filename)
    except FileNotFoundError:
        print(f"I guess this file {filename} was not found")
        
    return len(file.readlines())
```

More seasoned developers will know that your funtion should not simply hide exceptions. If you don't want to handle the exception, then you should let the calling function handle it:

```python
def count_lines(filename) throws FileNotFoundError: # <= Not actual Python syntax
    file = open(filename)
    return len(file.readlines())
```

The `throws` clause above is from Java and not actually part of Python. Being force to either handle the exception or two explicitely "throw" it and let the caller handle is called "Checked Exceptions." Most of the internet seems to hate this feature although the author of these lecture loves them. If you are using a typed language, the ability to know which exceptions can be raise by any number of lines in a function can be extremely helpful.